# Setup - Install Libraries

In [ ]:
# Run the following commands once, in order to install libraries - DO NOT Uncomment this line.

# Uncomment below lines

!pip3 install --upgrade pip
!pip install google-cloud-bigquery
!pip install pandas-gbq -U
!pip install db-dtypes
!pip install packaging --upgrade

# Import libraries

In [1]:
# Import libraries
from google.cloud import bigquery
import pandas as pd
from pandas_gbq import to_gbq
import os

print('Libraries imported successfully')

Libraries imported successfully


In [2]:
# Set the environment variable for Google Cloud credentials
# Place the path in which the .json file is located.

# Example (if .json is located in the same directory with the notebook)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "at-arch-416714-6f9900ec7.json"

# -- YOUR CODE GOES BELOW THIS LINE
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/Mouxios Giorgos/Downloads/Data Analytics/warm-influence-480112-t2-3027da7e0b9c.json" # Edit path
# -- YOUR CODE GOES ABOVE THIS LINE

In [3]:
# Set your Google Cloud project ID and BigQuery dataset details

# -- YOUR CODE GOES BELOW THIS

project_id = 'warm-influence-480112-t2' # Edit with your project id
dataset_id = 'staging_db' # Modify the necessary schema name: staging_db, reporting_db etc.
table_id = 'stg_film' # Modify the necessary table name: stg_customer, stg_city etc.

# -- YOUR CODE GOES ABOVE THIS LINE

# SQL Query

In [5]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

# -- YOUR CODE GOES BELOW THIS LINE

# Define your SQL query here
query ='''
with base as (
  select *
  from `warm-influence-480112-t2.pagila_production_mouxpublic.film`
),
final as (
  select
    film_id,
    title AS film_title,
    description AS film_description,
    language_id AS film_language_id,
    original_language_id AS film_original_language_id,
    rental_duration AS film_rental_duration,
    rental_rate AS film_rental_rate,
    length AS film_length,
    replacement_cost AS film_replacement_cost,
    rating AS film_rating,
    last_update AS film_last_update,
    special_features AS film_special_features,
    fulltext AS film_fulltext
  FROM base
)
select * from final
'''

# -- YOUR CODE GOES ABOVE THIS LINE

# Execute the query and store the result in a dataframe
df = client.query(query).to_dataframe()

# Explore some records
df.head()

C:\Users\Mouxios Giorgos\python312\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,film_id,film_title,film_description,film_language_id,film_original_language_id,film_rental_duration,film_rental_rate,film_length,film_replacement_cost,film_rating,film_last_update,film_special_features,film_fulltext
0,25,ANGELS LIFE,A Thoughtful Display of a Woman And a Astronau...,1,<NA>,3,2.990000000,74,15.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Trailers""]",'angel':1 'astronaut':11 'battl':14 'berlin':1...
1,402,HARPER DYING,A Awe-Inspiring Reflection of a Woman And a Ca...,1,<NA>,3,0.990000000,52,15.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Trailers""]",'awe':5 'awe-inspir':4 'cat':13 'confront':16 ...
2,653,PANIC CLUB,A Fanciful Display of a Teacher And a Crocodil...,1,<NA>,3,4.990000000,102,15.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Commentaries"",""Deleted Scenes""]",'baloon':19 'club':2 'crocodil':11 'display':5...
3,156,CLERKS ANGELS,A Thrilling Display of a Sumo Wrestler And a G...,1,<NA>,3,4.990000000,164,15.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Commentaries""]",'angel':2 'baloon':20 'clerk':1 'confront':15 ...
4,26,ANNIE IDENTITY,A Amazing Panorama of a Pastry Chef And a Boat...,1,<NA>,3,0.990000000,86,15.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Commentaries"",""Deleted Scenes""]",'abandon':20 'amaz':4 'amus':21 'anni':1 'boat...


# Write to BigQuery

In [9]:
# Define the full table ID
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# -- YOUR CODE GOES BELOW THIS LINE
# Define table schema based on the project description

schema = [
    bigquery.SchemaField('film_id', 'INTEGER'),
    bigquery.SchemaField('film_title', 'STRING'),
    bigquery.SchemaField('film_description', 'STRING'),
    bigquery.SchemaField('film_language_id', 'INTEGER'),
    bigquery.SchemaField('film_original_language_id', 'INTEGER'),
    bigquery.SchemaField('film_rental_duration', 'INTEGER'),
    bigquery.SchemaField('film_rental_rate', 'NUMERIC'),
    bigquery.SchemaField('film_length', 'INTEGER'),
    bigquery.SchemaField('film_replacement_cost', 'NUMERIC'),
    bigquery.SchemaField('film_rating', 'STRING'),
    bigquery.SchemaField('film_last_update', 'DATETIME'),
    bigquery.SchemaField('film_special_features', 'STRING'),
    bigquery.SchemaField('film_fulltext', 'STRING')
]


# -- YOUR CODE GOES ABOVE THIS LINE

In [10]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

# Check if the table exists
def table_exists(client, full_table_id):
    try:
        client.get_table(full_table_id)
        return True
    except Exception:
        return False

# Write the dataframe to the table (overwrite if it exists, create if it doesn't)
if table_exists(client, full_table_id):
    # If the table exists, overwrite it
    destination_table = f"{dataset_id}.{table_id}"
    # Write the dataframe to the table (overwrite if it exists)
    to_gbq(df, destination_table, project_id=project_id, if_exists='replace')
    print(f"Table {full_table_id} exists. Overwritten.")
else:
    # If the table does not exist, create it
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
    job.result()  # Wait for the job to complete
    print(f"Table {full_table_id} did not exist. Created and data loaded.")

Table warm-influence-480112-t2.staging_db.stg_film did not exist. Created and data loaded.
